In [4]:
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import numpy as np

In [315]:
url = "http://www.boattrader.com/search-results/NewOrUsed-any/Type-any/Category-all/Zip-14853/Radius-200/Sort-Length:DESC"
main_url = "http://www.boattrader.com" #/listing/2006-sea-ray-320-sundancer-103069603/

def string_check(value):
    if value is None:
        return "" 
    return str(value.text)

def from_table(soup,name):
    
    data = list()
    
    if(soup.find('div',{'id':name}) is not None):
        table = soup.find('div',{'id':name}).find('table')
        rows = table.find_all('tr')
        for row in rows:
            if(name =='measurements'):
                index = string_check(row.find('dt'))
                val  = string_check(row.find('dd')) 
            else:
                index = string_check(row.find('th'))
                val  = string_check(row.find('td'))
            
            data.append(val)
            
    return data
        

def getsoup(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r,'html.parser')
    return soup

def check_element(element):
    return element if element is not None else " "


def scrape_page(s):
    
    a = s.find("h1",attrs={"class":"bd-name"})
    b = s.find("div",attrs={"class":"bd-price-location"})
    data = []
    if a is not None:
        year =  str(check_element(a.find('span',{"class":"bd-year"})).get_text())
        make =  str(check_element(a.find('span',{"class":"bd-make"})).get_text())
        model = str(check_element(a.find('span',{"class":"bd-model"})).get_text())
        data.append(year)
        data.append(make)
        data.append(model)
        
    if b is not None:                
        price = str(check_element(b.find('span',{"class":"bd-price contact-toggle"})).get_text()).split()[0]
        data.append(price)

    content = s.find('div',{'class':'detail-tbl'})
    
    if content is not None:
        data  = data + from_table(content,'boat-details')
        data  = data + from_table(content,'measurements')
        data  = data + from_table(content,'propulsion')
        data  = data + from_table(content,'other-details')
                    
    print(data)
    return data

    
    
    
#     details = s.find('section',{"id":"collapsible-content-areas"}) 
#     details = s.find('section',{"id":"collapsible-content-areas"}) 
#     from_table(details,'boat-details')


In [316]:
soup = getsoup(url)
a = soup.find_all("a", attrs={"data-reporting-click-listing-type":"standard listing"})
links = [str(main_url+x['href']) for x in a]

data = list()

for link in links:
    s = getsoup(link)
    data.append(scrape_page(s))





['2006', 'Sea Ray', '320 Sundancer', '$99,995', 'Power', ' Cruisers', '2006', 'Sea Ray', "320'", 'Other', 'Other', 'Other', 'N Tonawanda, NY', '', '320', 'Other', 'Other', 'Other']
['2012', 'Stingray Boats', '215LR', '$27,900', 'Power', ' Deck Boats', '2012', 'Stingray Boats', "263'", 'Other', 'Other', 'Other', 'Tafton, PA', '', '263', 'Other', 'Other', 'Other']
['2016', 'Yamaha Sport Boat', 'SX240', '$45,999', 'Power', ' Jet Boats', '2016', 'Yamaha Sport Boat', "240'", 'Other', 'Fiberglass', '', 'Staten Island, NY', '', '240', '', '3605', 'Yamaha', 'Marine', 'Other', 'YB0189I', 'Deep Vee']
['1987', 'Blount', 'Luxury Dinner Yacht (GPC)', '$2,950,000', 'Power', ' Commercial Boats', '1987', 'Blount', "197'", 'Other', 'Steel', 'Diesel', 'Troy, NY', '', '197', 'Other', 'Diesel']
['2016', 'Yamaha Sport Boat', '190 FSH DELUXE', '$26,999', 'Power', ' Jet Boats, Bowrider', '2016', 'Yamaha Sport Boat', "190'", 'Other', 'Fiberglass', 'Gas', 'Staten Island, NY', '', '190', '', '2130', 'YAMAHA MAR

In [317]:
t = pd.DataFrame(data)

In [319]:
t.head(10)

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,2006,Sea Ray,320 Sundancer,"$99,995",Power,Cruisers,2006,Sea Ray,320',Other,...,None,None,None,None,None,None,None,None,None,None
1,2012,Stingray Boats,215LR,"$27,900",Power,Deck Boats,2012,Stingray Boats,263',Other,...,None,None,None,None,None,None,None,None,None,None
2,2016,Yamaha Sport Boat,SX240,"$45,999",Power,Jet Boats,2016,Yamaha Sport Boat,240',Other,...,Deep Vee,None,None,None,None,None,None,None,None,None
3,1987,Blount,Luxury Dinner Yacht (GPC),"$2,950,000",Power,Commercial Boats,1987,Blount,197',Other,...,None,None,None,None,None,None,None,None,None,None
4,2016,Yamaha Sport Boat,190 FSH DELUXE,"$26,999",Power,"Jet Boats, Bowrider",2016,Yamaha Sport Boat,190',Other,...,YB0273D,Deep Vee,None,None,None,None,None,None,None,None
5,2016,Yamaha Sport Boat,190 FSH SPORT,"$27,999",Power,Jet Boats,2016,Yamaha Sport Boat,190',Other,...,YB0095L,Deep Vee,None,None,None,None,None,None,None,None
6,2013,Aeroyacht,125,"$28,303,680",Sails,"Multi-Hulls, Catamaran",2013,Aeroyacht,125',Other,...,None,None,None,None,None,None,None,None,None,None
7,2012,Ab Yachts,116,"$14,000,000",Power,"Jet Boats, Mega Yachts",2012,Ab Yachts,116',Other,...,,6,MTU,16V2000 M93,7200hp,600,Other,Diesel,Planing,2 year
8,1999,Westport,Raised Pilothouse MY,"$3,950,000",Power,Motor Yachts,1999,Westport,112',Twin Inboard,...,3600hp,Twin Inboard,Diesel,None,None,None,None,None,None,None
9,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
